In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from datetime import timedelta
from utils import (
    prepare_data_prophet,
    split_data_prophet,
    train_model_prophet,
    make_forecast_prophet
)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tbb_path = r"C:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\tcc_2\Lib\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb"
os.environ["PATH"] += os.pathsep + tbb_path

# Favorita

In [3]:
file_path = 'Bases Tratadas/favorita_item_loja_semana.parquet'
df_favorita = pd.read_parquet(file_path)
display(df_favorita.head())

,Produto,Loja,Data,Vendas
0,96995,2,2013-01-07,2
33584,830624,15,2013-01-07,13
33585,830624,16,2013-01-07,52
33586,830624,17,2013-01-07,11
33587,830624,18,2013-01-07,4


In [4]:
df_favorita_prophet = prepare_data_prophet(df_favorita)
display(df_favorita_prophet.head())

,Produto,Loja,ds,y
0,96995,2,2013-01-07,2
33584,830624,15,2013-01-07,13
33585,830624,16,2013-01-07,52
33586,830624,17,2013-01-07,11
33587,830624,18,2013-01-07,4


## Todas as lojas e produtos

In [5]:
df_favorita_prophet_grouped = df_favorita_prophet.groupby('ds')['y'].sum().reset_index()
display(df_favorita_prophet_grouped.head())

,ds,y
0,2013-01-07,2546580
1,2013-01-14,2330352
2,2013-01-21,2384817
3,2013-01-28,2245352
4,2013-02-04,2534680


In [6]:
# Dividir o dataset em treino (80%) e teste (20%)
train, test = split_data_prophet(df_favorita_prophet_grouped)

# Calcular o tempo de treino
start_train_time = time.time()
model = train_model_prophet(train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Calcular o tempo de previsão
start_forecast_time = time.time()
forecast = make_forecast_prophet(model, periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Alinhar as previsões com os dados de teste
forecast_test = forecast.iloc[-len(test):].copy()
forecast_test['ds'] = test['ds'].values

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Exibir as previsões
fig = go.Figure()

# Add traces for train, test, and forecast
fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Train', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Test', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=forecast_test['ds'], y=forecast_test['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')))

# Add titles and labels
fig.update_layout(
    title='Sales Forecast',
    xaxis_title='Date',
    yaxis_title='Sales',
    legend_title='Legend',
    template='plotly_white'
)

# Show the figure
fig.show()


15:36:25 - cmdstanpy - INFO - Chain [1] start processing
15:36:25 - cmdstanpy - INFO - Chain [1] done processing


Tempo de treino: 0.387833833694458 segundos
Tempo de previsão: 0.12439632415771484 segundos


In [7]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test['ds'],
    'Real': test['y'],
    'Previsto': forecast_test['yhat']
})

# Salvar o dataframe em um arquivo CSV
display(df_resultado)
df_resultado.to_csv('Resultados/Previsao/Favorita/Tudo/prophet.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
display(df_tempo)
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Favorita/Tudo/prophet.csv', index=False)

,Data,Real,Previsto
168,2016-03-28,4922508,5.125992e+06
169,2016-04-04,5980373,4.937520e+06
170,2016-04-11,5346926,4.812489e+06
171,2016-04-18,6014831,4.716348e+06
172,2016-04-25,6818830,4.636214e+06
...,...,...,...
236,2017-07-17,5758675,7.328565e+06
237,2017-07-24,5738533,7.319774e+06
238,2017-07-31,5987120,6.979383e+06
239,2017-08-07,6317703,6.497020e+06


,Treino,Previsao
0,0.387834,0.124396


## Por lojas e produtos relevantes

In [8]:
lojas_relevantes_favorita = [44, 45, 47, 3, 49, 46, 48, 51, 8, 50]
produtos_relevantes_favorita = {
    1: [1503844, 1473474],
    2: [582864, 1503844],
    3: [1047679, 1503844],
    4: [1503844, 1473474],
    5: [584028, 1473474],
    6: [1503844, 1463992],
    7: [1503844, 1047679],
    8: [1503844, 1047679],
    9: [1463992, 457928],
    10: [584126, 1463992],
    11: [584126, 819932],
    12: [584126, 819932],
    13: [584126, 819932],
    14: [584126, 587157],
    15: [807493, 587157],
    16: [1167614, 1464086],
    17: [1463992, 1167614],
    18: [1695999, 857332],
    19: [819932, 584126],
    20: [1463992, 1463993],
    21: [1167614, 1464086],
    22: [1239882, 584126],
    23: [1084365, 583985],
    24: [257847, 1503844],
    25: [1146496, 257847],
    26: [584028, 583927],
    27: [1503844, 1503847],
    28: [1167614, 1464088],
    29: [1464086, 1167614],
    30: [1167614, 1463860],
    31: [1167614, 364606],
    32: [1167614, 812751],
    33: [1167614, 584032],
    34: [819932, 215352],
    35: [1167614, 1464086],
    36: [1464086, 587157],
    37: [1503844, 1463864],
    38: [502331, 1463860],
    39: [1167614, 1463864],
    40: [1463855, 1463860],
    41: [559870, 1503844],
    42: [1463864, 1503844],
    43: [807493, 1143685],
    44: [1047679, 1503844],
    45: [1503844, 1473474],
    46: [584028, 1463992],
    47: [1503844, 1047679],
    48: [584028, 1463992],
    49: [1503844, 1473474],
    50: [584028, 364606],
    51: [1503844, 257847],
    52: [1503844, 1473474],
    53: [1473474, 1503844],
    54: [1464210, 1143685]
}

In [9]:
# Dicionário para armazenar os modelos treinados para cada loja
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_favorita])

# Criar um dataframe para armazenar os resultados
df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for i, loja in enumerate(lojas_relevantes_favorita):
    # Filtrar os dados para a loja atual
    df_loja = df_favorita_prophet[df_favorita_prophet['Loja'] == loja]
    
    # Agrupar os dados por data e somar as vendas
    df_loja_grouped = df_loja.groupby('ds')['y'].sum().reset_index()
    
    # Dividir os dados em treino e teste
    train_loja, test_loja = split_data_prophet(df_loja_grouped)
    
    # Calcular o tempo de treino
    start_train_time = time.time()
    model_loja = train_model_prophet(train_loja)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    
    # Calcular o tempo de previsão
    start_forecast_time = time.time()
    forecast = make_forecast_prophet(model_loja, periods=len(test_loja))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time
    
    # Alinhar as previsões com os dados de teste
    forecast_test_loja = forecast.iloc[-len(test_loja):].copy()
    forecast_test_loja['ds'] = test_loja['ds'].values
    
    # Armazenar o modelo treinado no dicionário
    modelos_por_loja[loja] = model_loja
    
    # Adicionar os resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_loja['ds'],
        'Loja': loja,
        'Real': test_loja['y'],
        'Previsto': forecast_test_loja['yhat']
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)
    
    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_loja['ds'], y=train_loja['y'], mode='lines', name='Train', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_loja['ds'], y=test_loja['y'], mode='lines', name='Test', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=forecast_test_loja['ds'], y=forecast_test_loja['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Date', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Sales', row=i//2+1, col=i%2+1)
    # Criar um dataframe com o tempo de treino e previsão por loja



# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/Favorita/Loja/prophet.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Favorita/Loja/prophet.csv', index=False)
# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)


15:36:27 - cmdstanpy - INFO - Chain [1] start processing
15:36:27 - cmdstanpy - INFO - Chain [1] done processing
15:36:28 - cmdstanpy - INFO - Chain [1] start processing
15:36:28 - cmdstanpy - INFO - Chain [1] done processing
15:36:28 - cmdstanpy - INFO - Chain [1] start processing
15:36:28 - cmdstanpy - INFO - Chain [1] done processing
15:36:28 - cmdstanpy - INFO - Chain [1] start processing
15:36:29 - cmdstanpy - INFO - Chain [1] done processing
15:36:29 - cmdstanpy - INFO - Chain [1] start processing
15:36:29 - cmdstanpy - INFO - Chain [1] done processing
15:36:29 - cmdstanpy - INFO - Chain [1] start processing
15:36:29 - cmdstanpy - INFO - Chain [1] done processing
15:36:30 - cmdstanpy - INFO - Chain [1] start processing
15:36:30 - cmdstanpy - INFO - Chain [1] done processing
15:36:30 - cmdstanpy - INFO - Chain [1] start processing
15:36:30 - cmdstanpy - INFO - Chain [1] done processing
15:36:30 - cmdstanpy - INFO - Chain [1] start processing
15:36:30 - cmdstanpy - INFO - Chain [1]

Tempos de treinamento por loja: {44: 0.22809934616088867, 45: 0.23245716094970703, 47: 0.22675228118896484, 3: 0.25005125999450684, 49: 0.22144317626953125, 46: 0.19517016410827637, 48: 0.18585681915283203, 51: 0.17483091354370117, 8: 0.18386340141296387, 50: 0.18613982200622559}
Tempos de previsão por loja: {44: 0.18139100074768066, 45: 0.10491609573364258, 47: 0.10512018203735352, 3: 0.06858682632446289, 49: 0.09339547157287598, 46: 0.05699634552001953, 48: 0.052401065826416016, 51: 0.06555366516113281, 8: 0.08193397521972656, 50: 0.1275336742401123}


In [10]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_favorita:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_favorita.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        # Filtrar os dados para a loja e item atuais
        df_loja_item = df_favorita_prophet[(df_favorita_prophet['Loja'] == loja) & (df_favorita_prophet['Produto'] == item)]
        
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar os dados por data e somar as vendas
        df_loja_item_grouped = df_loja_item.groupby('ds')['y'].sum().reset_index()
        
        # Dividir os dados em treino e teste
        train_loja_item, test_loja_item = split_data_prophet(df_loja_item_grouped)
        
        # Calcular o tempo de treino
        start_train_time = time.time()
        model_loja_item = train_model_prophet(train_loja_item)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time
        tempos_treinamento[(loja, item)] = train_time
        
        # Calcular o tempo de previsão
        start_forecast_time = time.time()
        forecast = make_forecast_prophet(model_loja_item, periods=len(test_loja_item))
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time
        tempos_previsao[(loja, item)] = forecast_time
        
        # Alinhar as previsões com os dados de teste
        forecast_test_loja_item = forecast.iloc[-len(test_loja_item):].copy()
        forecast_test_loja_item['ds'] = test_loja_item['ds'].values
        
        # Armazenar o modelo treinado no dicionário
        modelos_por_loja_item[(loja, item)] = model_loja_item
        
        # Adicionar os resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_loja_item['ds'],
            'Loja': loja,
            'Item': item,
            'Real': test_loja_item['y'],
            'Previsto': forecast_test_loja_item['yhat']
        })])
        
        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)
        
        # Adicionar traces para treino, teste e previsão no gráfico da loja
        fig_loja.add_trace(go.Scatter(
            x=train_loja_item['ds'], y=train_loja_item['y'],
            mode='lines', name=f'Train (Item {item})', line=dict(color='blue')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=test_loja_item['ds'], y=test_loja_item['y'],
            mode='lines', name=f'Test (Item {item})', line=dict(color='orange')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=forecast_test_loja_item['ds'], y=forecast_test_loja_item['yhat'],
            mode='lines', name=f'Forecast (Item {item})', line=dict(color='green')
        ), row=idx+1, col=1)
        
        # Atualizar layout para cada subplot
        fig_loja.update_xaxes(title_text='Date', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Sales', row=idx+1, col=1)
    
    # Atualizar layout para a figura completa da loja
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),  # Ajusta o tamanho de acordo com a quantidade de itens
        showlegend=False,
        template='plotly_white'
    )
    
    # Salvar e exibir o gráfico para a loja
    fig_loja.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Favorita/Item/prophet.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Favorita/Item/prophet.csv', index=False)

# Imprimir tempos de treinamento e previsão
print("Tempos de treinamento por loja e item:", tempos_treinamento)
print("Tempos de previsão por loja e item:", tempos_previsao)



15:36:31 - cmdstanpy - INFO - Chain [1] start processing
15:36:31 - cmdstanpy - INFO - Chain [1] done processing
15:36:32 - cmdstanpy - INFO - Chain [1] start processing
15:36:32 - cmdstanpy - INFO - Chain [1] done processing


15:36:32 - cmdstanpy - INFO - Chain [1] start processing
15:36:32 - cmdstanpy - INFO - Chain [1] done processing
15:36:32 - cmdstanpy - INFO - Chain [1] start processing
15:36:32 - cmdstanpy - INFO - Chain [1] done processing


15:36:32 - cmdstanpy - INFO - Chain [1] start processing
15:36:32 - cmdstanpy - INFO - Chain [1] done processing
15:36:33 - cmdstanpy - INFO - Chain [1] start processing
15:36:33 - cmdstanpy - INFO - Chain [1] done processing


15:36:33 - cmdstanpy - INFO - Chain [1] start processing
15:36:33 - cmdstanpy - INFO - Chain [1] done processing
15:36:33 - cmdstanpy - INFO - Chain [1] start processing
15:36:33 - cmdstanpy - INFO - Chain [1] done processing


15:36:34 - cmdstanpy - INFO - Chain [1] start processing
15:36:34 - cmdstanpy - INFO - Chain [1] done processing
15:36:34 - cmdstanpy - INFO - Chain [1] start processing
15:36:34 - cmdstanpy - INFO - Chain [1] done processing


...

15:36:35 - cmdstanpy - INFO - Chain [1] start processing
15:36:35 - cmdstanpy - INFO - Chain [1] done processing
15:36:35 - cmdstanpy - INFO - Chain [1] start processing
15:36:35 - cmdstanpy - INFO - Chain [1] done processing


15:36:36 - cmdstanpy - INFO - Chain [1] start processing
15:36:36 - cmdstanpy - INFO - Chain [1] done processing
15:36:36 - cmdstanpy - INFO - Chain [1] start processing
15:36:36 - cmdstanpy - INFO - Chain [1] done processing


15:36:36 - cmdstanpy - INFO - Chain [1] start processing
15:36:36 - cmdstanpy - INFO - Chain [1] done processing
15:36:37 - cmdstanpy - INFO - Chain [1] start processing
15:36:37 - cmdstanpy - INFO - Chain [1] done processing


15:36:37 - cmdstanpy - INFO - Chain [1] start processing
15:36:37 - cmdstanpy - INFO - Chain [1] done processing
15:36:38 - cmdstanpy - INFO - Chain [1] start processing
15:36:38 - cmdstanpy - INFO - Chain [1] done processing


Tempos de treinamento por loja e item: {(44, 1047679): 0.19695425033569336, (44, 1503844): 0.1745133399963379, (45, 1503844): 0.16789031028747559, (45, 1473474): 0.17042040824890137, (47, 1503844): 0.15210390090942383, (47, 1047679): 0.17398381233215332, (3, 1047679): 0.1672215461730957, (3, 1503844): 0.18133211135864258, (49, 1503844): 0.16761541366577148, (49, 1473474): 0.17326784133911133, (46, 584028): 0.16745495796203613, (46, 1463992): 0.15039443969726562, (48, 584028): 0.17426824569702148, (48, 1463992): 0.23363423347473145, (51, 1503844): 0.18235039710998535, (51, 257847): 0.18874645233154297, (8, 1503844): 0.19808745384216309, (8, 1047679): 0.2336711883544922, (50, 584028): 0.2300121784210205, (50, 364606): 0.21668267250061035}
Tempos de previsão por loja e item: {(44, 1047679): 0.06757235527038574, (44, 1503844): 0.04192781448364258, (45, 1503844): 0.04770207405090332, (45, 1473474): 0.04772782325744629, (47, 1503844): 0.06312894821166992, (47, 1047679): 0.04792332649230957, 

# M5

In [11]:
file_path_m5 = 'Bases Tratadas/m5_item_loja_semana.parquet'
df_m5 = pd.read_parquet(file_path_m5)
display(df_m5.head())

,Produto,Loja,Data,Vendas
0,HOBBIES_1_001,CA_1,1,0
1,HOBBIES_1_002,CA_1,1,0
2,HOBBIES_1_003,CA_1,1,0
3,HOBBIES_1_004,CA_1,1,0
4,HOBBIES_1_005,CA_1,1,0


In [12]:
df_m5_prophet = prepare_data_prophet(df_m5)
display(df_m5_prophet.head())

,Produto,Loja,ds,y
0,HOBBIES_1_001,CA_1,1,0
1,HOBBIES_1_002,CA_1,1,0
2,HOBBIES_1_003,CA_1,1,0
3,HOBBIES_1_004,CA_1,1,0
4,HOBBIES_1_005,CA_1,1,0


In [13]:
data_inicial = pd.to_datetime('2018-01-01')

# Atualize a coluna 'ds' com datas calculadas a partir das semanas
df_m5_prophet['ds'] = data_inicial + pd.to_timedelta((df_m5_prophet['ds'] - 1) * 7, unit='D')

In [14]:
display(df_m5_prophet)

,Produto,Loja,ds,y
0,HOBBIES_1_001,CA_1,2018-01-01,0
1,HOBBIES_1_002,CA_1,2018-01-01,0
2,HOBBIES_1_003,CA_1,2018-01-01,0
3,HOBBIES_1_004,CA_1,2018-01-01,0
4,HOBBIES_1_005,CA_1,2018-01-01,0
...,...,...,...,...
59089615,FOODS_3_823,WI_3,2023-04-17,0
59089616,FOODS_3_824,WI_3,2023-04-17,1
59089617,FOODS_3_825,WI_3,2023-04-17,0
59089618,FOODS_3_826,WI_3,2023-04-17,1


## Todas as lojas e produtos

In [15]:
df_m5_prophet_grouped = df_m5_prophet.groupby('ds')['y'].sum().reset_index()
display(df_m5_prophet_grouped.head())

,ds,y
0,2018-01-01,161932
1,2018-01-08,194735
2,2018-01-15,185293
3,2018-01-22,170194
4,2018-01-29,170777


In [16]:
# Dividir o dataset em treino (80%) e teste (20%)
train, test = split_data_prophet(df_m5_prophet_grouped)

# Calcular o tempo de treino
start_train_time = time.time()
model = train_model_prophet(train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Calcular o tempo de previsão
start_forecast_time = time.time()
forecast = make_forecast_prophet(model, periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Alinhar as previsões com os dados de teste
forecast_test = forecast.iloc[-len(test):].copy()
forecast_test['ds'] = test['ds'].values

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Exibir as previsões
fig = go.Figure()

# Add traces for train, test, and forecast
fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Train', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Test', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=forecast_test['ds'], y=forecast_test['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')))

# Add titles and labels
fig.update_layout(
    title='Sales Forecast',
    xaxis_title='Date',
    yaxis_title='Sales',
    legend_title='Legend',
    template='plotly_white'
)

# Show the figure
fig.show()

15:36:55 - cmdstanpy - INFO - Chain [1] start processing
15:36:55 - cmdstanpy - INFO - Chain [1] done processing


Tempo de treino: 0.2888150215148926 segundos
Tempo de previsão: 0.10093092918395996 segundos


In [ ]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test['ds'],
    'Real': test['y'],
    'Previsto': forecast_test['yhat']
})

# Salvar o dataframe em um arquivo CSV
display(df_resultado)
df_resultado.to_csv('Resultados/Previsao/M5/Tudo/prophet.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
display(df_tempo)
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/M5/Tudo/prophet.csv', index=False)

,Data,Real,Previsto
193,2021-09-13,256434,253796.453518
194,2021-09-20,238673,249347.120261
195,2021-09-27,231081,247529.505706
196,2021-10-04,257742,247061.369217
197,2021-10-11,252686,245428.428130
...,...,...,...
272,2023-03-20,287796,250960.425471
273,2023-03-27,282731,246978.449606
274,2023-04-03,314216,244509.776006
275,2023-04-10,312183,243637.450234


,Treino,Previsao
0,0.288815,0.100931


## Por loja e produto

In [18]:
lojas_relevantes_m5 = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

produtos_relevantes_m5 = {
    'CA_1': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_2': ['FOODS_3_586', 'FOODS_3_252'],
    'CA_3': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_4': ['FOODS_3_090', 'FOODS_3_586'],
    'TX_1': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_2': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_3': ['FOODS_3_586', 'FOODS_3_090'],
    'WI_1': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_2': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_3': ['FOODS_3_090', 'FOODS_3_226']
}

In [19]:
# Dicionário para armazenar os modelos treinados para cada loja
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_m5])

# Criar um dataframe para armazenar os resultados
df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for i, loja in enumerate(lojas_relevantes_m5):
    # Filtrar os dados para a loja atual
    df_loja = df_m5_prophet[df_m5_prophet['Loja'] == loja]
    
    # Agrupar os dados por data e somar as vendas
    df_loja_grouped = df_loja.groupby('ds')['y'].sum().reset_index()
    
    # Dividir os dados em treino e teste
    train_loja, test_loja = split_data_prophet(df_loja_grouped)
    
    # Calcular o tempo de treino
    start_train_time = time.time()
    model_loja = train_model_prophet(train_loja)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    
    # Calcular o tempo de previsão
    start_forecast_time = time.time()
    forecast = make_forecast_prophet(model_loja, periods=len(test_loja))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time
    
    # Alinhar as previsões com os dados de teste
    forecast_test_loja = forecast.iloc[-len(test_loja):].copy()
    forecast_test_loja['ds'] = test_loja['ds'].values
    
    # Armazenar o modelo treinado no dicionário
    modelos_por_loja[loja] = model_loja
    
    # Adicionar os resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_loja['ds'],
        'Loja': loja,
        'Real': test_loja['y'],
        'Previsto': forecast_test_loja['yhat']
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)
    
    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_loja['ds'], y=train_loja['y'], mode='lines', name='Train', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_loja['ds'], y=test_loja['y'], mode='lines', name='Test', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=forecast_test_loja['ds'], y=forecast_test_loja['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Date', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Sales', row=i//2+1, col=i%2+1)
    # Criar um dataframe com o tempo de treino e previsão por loja



# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/M5/Loja/prophet.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/M5/Loja/prophet.csv', index=False)
# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)

15:37:00 - cmdstanpy - INFO - Chain [1] start processing
15:37:00 - cmdstanpy - INFO - Chain [1] done processing
15:37:03 - cmdstanpy - INFO - Chain [1] start processing
15:37:03 - cmdstanpy - INFO - Chain [1] done processing
15:37:06 - cmdstanpy - INFO - Chain [1] start processing
15:37:06 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:14 - cmdstanpy - INFO - Chain [1] start processing
15:37:15 - cmdstanpy - INFO - Chain [1] done processing
15:37:19 - cmdstanpy - INFO - Chain [1] start processing
15:37:19 - cmdstanpy - INFO - Chain [1] done processing
15:37:22 - cmdstanpy - INFO - Chain [1] start processing
15:37:22 - cmdstanpy - INFO - Chain [1] done processing
15:37:26 - cmdstanpy - INFO - Chain [1] start processing
15:37:26 - cmdstanpy - INFO - Chain [1] done processing
15:37:29 - cmdstanpy - INFO - Chain [1] start processing
15:37:30 - cmdstanpy - INFO - Chain [1]

Tempos de treinamento por loja: {'CA_1': 0.1781902313232422, 'CA_2': 0.22265362739562988, 'CA_3': 0.19035100936889648, 'CA_4': 0.20605874061584473, 'TX_1': 0.2588460445404053, 'TX_2': 0.20423364639282227, 'TX_3': 0.1995539665222168, 'WI_1': 0.1840195655822754, 'WI_2': 0.20962738990783691, 'WI_3': 0.16402149200439453}
Tempos de previsão por loja: {'CA_1': 0.06042075157165527, 'CA_2': 0.07936906814575195, 'CA_3': 0.08650016784667969, 'CA_4': 0.0780634880065918, 'TX_1': 0.10030484199523926, 'TX_2': 0.08086895942687988, 'TX_3': 0.060137033462524414, 'WI_1': 0.10026788711547852, 'WI_2': 0.07307720184326172, 'WI_3': 0.06752276420593262}


In [20]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_m5:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_m5.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        # Filtrar os dados para a loja e item atuais
        df_loja_item = df_m5_prophet[(df_m5_prophet['Loja'] == loja) & (df_m5_prophet['Produto'] == item)]
        
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar os dados por data e somar as vendas
        df_loja_item_grouped = df_loja_item.groupby('ds')['y'].sum().reset_index()
        
        # Dividir os dados em treino e teste
        train_loja_item, test_loja_item = split_data_prophet(df_loja_item_grouped)
        
        # Calcular o tempo de treino
        start_train_time = time.time()
        model_loja_item = train_model_prophet(train_loja_item)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time
        tempos_treinamento[(loja, item)] = train_time
        
        # Calcular o tempo de previsão
        start_forecast_time = time.time()
        forecast = make_forecast_prophet(model_loja_item, periods=len(test_loja_item))
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time
        tempos_previsao[(loja, item)] = forecast_time
        
        # Alinhar as previsões com os dados de teste
        forecast_test_loja_item = forecast.iloc[-len(test_loja_item):].copy()
        forecast_test_loja_item['ds'] = test_loja_item['ds'].values
        
        # Armazenar o modelo treinado no dicionário
        modelos_por_loja_item[(loja, item)] = model_loja_item
        
        # Adicionar os resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_loja_item['ds'],
            'Loja': loja,
            'Item': item,
            'Real': test_loja_item['y'],
            'Previsto': forecast_test_loja_item['yhat']
        })])
        
        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)
        
        # Adicionar traces para treino, teste e previsão no gráfico da loja
        fig_loja.add_trace(go.Scatter(
            x=train_loja_item['ds'], y=train_loja_item['y'],
            mode='lines', name=f'Train (Item {item})', line=dict(color='blue')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=test_loja_item['ds'], y=test_loja_item['y'],
            mode='lines', name=f'Test (Item {item})', line=dict(color='orange')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=forecast_test_loja_item['ds'], y=forecast_test_loja_item['yhat'],
            mode='lines', name=f'Forecast (Item {item})', line=dict(color='green')
        ), row=idx+1, col=1)
        
        # Atualizar layout para cada subplot
        fig_loja.update_xaxes(title_text='Date', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Sales', row=idx+1, col=1)
    
    # Atualizar layout para a figura completa da loja
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),  # Ajusta o tamanho de acordo com a quantidade de itens
        showlegend=False,
        template='plotly_white'
    )
    
    # Salvar e exibir o gráfico para a loja
    fig_loja.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/M5/Item/prophet.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/M5/Item/prophet.csv', index=False)

# Imprimir tempos de treinamento e previsão
print("Tempos de treinamento por loja e item:", tempos_treinamento)
print("Tempos de previsão por loja e item:", tempos_previsao)



15:37:39 - cmdstanpy - INFO - Chain [1] start processing
15:37:39 - cmdstanpy - INFO - Chain [1] done processing
15:37:45 - cmdstanpy - INFO - Chain [1] start processing
15:37:45 - cmdstanpy - INFO - Chain [1] done processing


15:37:51 - cmdstanpy - INFO - Chain [1] start processing
15:37:51 - cmdstanpy - INFO - Chain [1] done processing
15:37:58 - cmdstanpy - INFO - Chain [1] start processing
15:37:58 - cmdstanpy - INFO - Chain [1] done processing


15:38:05 - cmdstanpy - INFO - Chain [1] start processing
15:38:05 - cmdstanpy - INFO - Chain [1] done processing
15:38:11 - cmdstanpy - INFO - Chain [1] start processing
15:38:11 - cmdstanpy - INFO - Chain [1] done processing


15:38:17 - cmdstanpy - INFO - Chain [1] start processing
15:38:17 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing


15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:38 - cmdstanpy - INFO - Chain [1] start processing
15:38:38 - cmdstanpy - INFO - Chain [1] done processing


...

15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:10 - cmdstanpy - INFO - Chain [1] start processing
15:39:10 - cmdstanpy - INFO - Chain [1] done processing


15:39:16 - cmdstanpy - INFO - Chain [1] start processing
15:39:16 - cmdstanpy - INFO - Chain [1] done processing
15:39:23 - cmdstanpy - INFO - Chain [1] start processing
15:39:23 - cmdstanpy - INFO - Chain [1] done processing


15:39:29 - cmdstanpy - INFO - Chain [1] start processing
15:39:29 - cmdstanpy - INFO - Chain [1] done processing
15:39:36 - cmdstanpy - INFO - Chain [1] start processing
15:39:36 - cmdstanpy - INFO - Chain [1] done processing


15:39:42 - cmdstanpy - INFO - Chain [1] start processing
15:39:43 - cmdstanpy - INFO - Chain [1] done processing
15:39:49 - cmdstanpy - INFO - Chain [1] start processing
15:39:49 - cmdstanpy - INFO - Chain [1] done processing


Tempos de treinamento por loja e item: {('CA_1', 'FOODS_3_090'): 0.20333433151245117, ('CA_1', 'FOODS_3_586'): 0.19443821907043457, ('CA_2', 'FOODS_3_586'): 0.18841004371643066, ('CA_2', 'FOODS_3_252'): 0.20063257217407227, ('CA_3', 'FOODS_3_090'): 0.19713401794433594, ('CA_3', 'FOODS_3_586'): 0.1826319694519043, ('CA_4', 'FOODS_3_090'): 0.18342876434326172, ('CA_4', 'FOODS_3_586'): 0.287564754486084, ('TX_1', 'FOODS_3_586'): 0.24932527542114258, ('TX_1', 'FOODS_3_090'): 0.18462920188903809, ('TX_2', 'FOODS_3_586'): 0.20277190208435059, ('TX_2', 'FOODS_3_090'): 0.3549063205718994, ('TX_3', 'FOODS_3_586'): 0.1549234390258789, ('TX_3', 'FOODS_3_090'): 0.19893860816955566, ('WI_1', 'FOODS_3_226'): 0.27069616317749023, ('WI_1', 'FOODS_3_694'): 0.25060582160949707, ('WI_2', 'FOODS_3_226'): 0.20865917205810547, ('WI_2', 'FOODS_3_694'): 0.21753430366516113, ('WI_3', 'FOODS_3_090'): 0.18987178802490234, ('WI_3', 'FOODS_3_226'): 0.2084343433380127}
Tempos de previsão por loja e item: {('CA_1', 

# Olist

In [21]:
file_path_olist = 'Bases Tratadas/olist_item_loja_semana.parquet'
df_olist = pd.read_parquet(file_path_olist)
display(df_olist.head())

,Produto,Loja,Data,Vendas
0,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
1,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
2,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-05,1
3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19,3
4,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,2016-10-03,1


In [22]:
df_olist_prophet = prepare_data_prophet(df_olist)
display(df_olist_prophet.head())

,Produto,Loja,ds,y
0,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
1,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
2,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-05,1
3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19,3
4,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,2016-10-03,1


## Todas as lojas e produtos

In [23]:
df_m5_olist_grouped = df_olist_prophet.groupby('ds')['y'].sum().reset_index()
display(df_m5_olist_grouped.head())

,ds,y
0,2016-09-05,3
1,2016-09-19,3
2,2016-10-03,9
3,2016-10-10,354
4,2016-12-26,1


In [24]:
# Dividir o dataset em treino (80%) e teste (20%)
train, test = split_data_prophet(df_m5_olist_grouped)

# Calcular o tempo de treino
start_train_time = time.time()
model = train_model_prophet(train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Calcular o tempo de previsão
start_forecast_time = time.time()
forecast = make_forecast_prophet(model, periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Alinhar as previsões com os dados de teste
forecast_test = forecast.iloc[-len(test):].copy()
forecast_test['ds'] = test['ds'].values

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Exibir as previsões
fig = go.Figure()

# Add traces for train, test, and forecast
fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Train', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Test', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=forecast_test['ds'], y=forecast_test['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')))

# Add titles and labels
fig.update_layout(
    title='Sales Forecast',
    xaxis_title='Date',
    yaxis_title='Sales',
    legend_title='Legend',
    template='plotly_white'
)

# Show the figure
fig.show()

15:39:49 - cmdstanpy - INFO - Chain [1] start processing
15:39:50 - cmdstanpy - INFO - Chain [1] done processing


Tempo de treino: 0.4389326572418213 segundos
Tempo de previsão: 0.0449373722076416 segundos


In [25]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test['ds'],
    'Real': test['y'],
    'Previsto': forecast_test['yhat']
})

# Salvar o dataframe em um arquivo CSV
display(df_resultado)
df_resultado.to_csv('Resultados/Previsao/Olist/Tudo/prophet.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
display(df_tempo)
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Olist/Tudo/prophet.csv', index=False)

,Data,Real,Previsto
63,2018-02-19,1922,1734.365504
64,2018-02-26,2059,1638.380673
65,2018-03-05,2139,1721.311504
66,2018-03-12,1808,2066.632805
67,2018-03-19,1911,2432.002639
68,2018-03-26,1950,2461.792852
69,2018-04-02,1697,2090.154633
70,2018-04-09,1758,1649.515651
71,2018-04-16,1837,1535.416083
72,2018-04-23,1941,1809.206196


,Treino,Previsao
0,0.438933,0.044937


## Por loja e produto

In [26]:
# Agrupar os dados por loja e calcular a soma das vendas
lojas_relevantes_olist = df_olist.groupby('Loja')['Vendas'].sum().sort_values(ascending=False).head(10).index.tolist()

# Dicionário para armazenar os produtos mais relevantes de cada loja
produtos_relevantes_olist = {}

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Filtrar os dados para a loja atual
    df_loja = df_olist[df_olist['Loja'] == loja]
    
    # Agrupar os dados por produto e calcular a soma das vendas
    produtos_relevantes_loja = df_loja.groupby('Produto')['Vendas'].sum().sort_values(ascending=False).head(2).index.tolist()
    
    # Armazenar os produtos mais relevantes no dicionário
    produtos_relevantes_olist[loja] = produtos_relevantes_loja

# Exibir as lojas e produtos relevantes
print("Lojas relevantes:", lojas_relevantes_olist)
print("Produtos relevantes por loja:", produtos_relevantes_olist)



Lojas relevantes: ['6560211a19b47992c3666cc44a7e94c0', '4a3ca9315b744ce9f8e9374361493884', '1f50f920176fa81dab994f9023523100', 'cc419e0650a3c5ba77189a1882b7556a', 'da8622b14eb17ae2831f4ac5b9dab84a', '955fee9216a65b617aa5c0531780ce60', '1025f0e2d44d7041d6cf58b6550e0bfa', '7c67e1448b00f6e969d365cea6b010ab', 'ea8482cd71df3c1969d7b9473ff13abc', '7a67c85e85bb2ce8582c35f2203ad736']
Produtos relevantes por loja: {'6560211a19b47992c3666cc44a7e94c0': ['a92930c327948861c015c919a0bcb4a8', 'd017a2151d543a9885604dc62a3d9dcc'], '4a3ca9315b744ce9f8e9374361493884': ['99a4788cb24856965c36a24e339b6058', '35afc973633aaeb6b877ff57b2793310'], '1f50f920176fa81dab994f9023523100': ['422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b'], 'cc419e0650a3c5ba77189a1882b7556a': ['154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'], 'da8622b14eb17ae2831f4ac5b9dab84a': ['e0cf79767c5b016251fe139915c59a26', 'fc1d8637c0268af3db482c14b7ef8e75'], '955fee9216a65b617aa5c0531780ce60': ['ac

In [27]:
# Dicionário para armazenar os modelos treinados para cada loja
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_olist])

# Criar um dataframe para armazenar os resultados
df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for i, loja in enumerate(lojas_relevantes_olist):
    # Filtrar os dados para a loja atual
    df_loja = df_olist_prophet[df_olist_prophet['Loja'] == loja]
    
    # Agrupar os dados por data e somar as vendas
    df_loja_grouped = df_loja.groupby('ds')['y'].sum().reset_index()
    date_range = pd.date_range(start=df_olist_prophet['ds'].min(), end=df_olist_prophet['ds'].max(), freq='W')
    date_range = date_range + pd.DateOffset(days=1)
    df_loja_grouped = df_loja_grouped.set_index('ds').reindex(date_range, fill_value=0).rename_axis('ds').reset_index()

    # Dividir os dados em treino e teste
    train_loja, test_loja = split_data_prophet(df_loja_grouped)
    
    # Calcular o tempo de treino
    start_train_time = time.time()
    model_loja = train_model_prophet(train_loja)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    
    # Calcular o tempo de previsão
    start_forecast_time = time.time()
    forecast = make_forecast_prophet(model_loja, periods=len(test_loja))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time
    
    # Alinhar as previsões com os dados de teste
    forecast_test_loja = forecast.iloc[-len(test_loja):].copy()
    forecast_test_loja['ds'] = test_loja['ds'].values
    
    # Armazenar o modelo treinado no dicionário
    modelos_por_loja[loja] = model_loja
    
    # Adicionar os resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_loja['ds'],
        'Loja': loja,
        'Real': test_loja['y'],
        'Previsto': forecast_test_loja['yhat']
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)
    
    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_loja['ds'], y=train_loja['y'], mode='lines', name='Train', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_loja['ds'], y=test_loja['y'], mode='lines', name='Test', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=forecast_test_loja['ds'], y=forecast_test_loja['yhat'], mode='lines', name='Forecast Test', line=dict(color='green')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Date', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Sales', row=i//2+1, col=i%2+1)
    # Criar um dataframe com o tempo de treino e previsão por loja



# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/Olist/Loja/prophet.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Olist/Loja/prophet.csv', index=False)
# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)

15:39:50 - cmdstanpy - INFO - Chain [1] start processing
15:39:51 - cmdstanpy - INFO - Chain [1] done processing
15:39:51 - cmdstanpy - INFO - Chain [1] start processing
15:39:51 - cmdstanpy - INFO - Chain [1] done processing
15:39:52 - cmdstanpy - INFO - Chain [1] start processing
15:39:52 - cmdstanpy - INFO - Chain [1] done processing
15:39:52 - cmdstanpy - INFO - Chain [1] start processing
15:39:52 - cmdstanpy - INFO - Chain [1] done processing
15:39:53 - cmdstanpy - INFO - Chain [1] start processing
15:39:53 - cmdstanpy - INFO - Chain [1] done processing
15:39:53 - cmdstanpy - INFO - Chain [1] start processing
15:39:54 - cmdstanpy - INFO - Chain [1] done processing
15:39:54 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:56 - cmdstanpy - INFO - Chain [1]

Tempos de treinamento por loja: {'6560211a19b47992c3666cc44a7e94c0': 0.6874034404754639, '4a3ca9315b744ce9f8e9374361493884': 0.5512192249298096, '1f50f920176fa81dab994f9023523100': 0.45169806480407715, 'cc419e0650a3c5ba77189a1882b7556a': 0.5261921882629395, 'da8622b14eb17ae2831f4ac5b9dab84a': 0.5040535926818848, '955fee9216a65b617aa5c0531780ce60': 0.7976539134979248, '1025f0e2d44d7041d6cf58b6550e0bfa': 0.64042067527771, '7c67e1448b00f6e969d365cea6b010ab': 0.3629934787750244, 'ea8482cd71df3c1969d7b9473ff13abc': 0.5684711933135986, '7a67c85e85bb2ce8582c35f2203ad736': 0.396730899810791}
Tempos de previsão por loja: {'6560211a19b47992c3666cc44a7e94c0': 0.041712045669555664, '4a3ca9315b744ce9f8e9374361493884': 0.03429150581359863, '1f50f920176fa81dab994f9023523100': 0.05008745193481445, 'cc419e0650a3c5ba77189a1882b7556a': 0.0424807071685791, 'da8622b14eb17ae2831f4ac5b9dab84a': 0.040824174880981445, '955fee9216a65b617aa5c0531780ce60': 0.028189897537231445, '1025f0e2d44d7041d6cf58b6550e0bfa':

In [28]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_olist.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        # Filtrar os dados para a loja e item atuais
        df_loja_item = df_olist_prophet[(df_olist_prophet['Loja'] == loja) & (df_olist_prophet['Produto'] == item)]
        
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar os dados por data e somar as vendas
        df_loja_item_grouped = df_loja_item.groupby('ds')['y'].sum().reset_index()
        date_range = pd.date_range(start=df_olist_prophet['ds'].min(), end=df_olist_prophet['ds'].max(), freq='W')
        date_range = date_range + pd.DateOffset(days=1)
        df_loja_item_grouped = df_loja_item_grouped.set_index('ds').reindex(date_range, fill_value=0).rename_axis('ds').reset_index()
        # Dividir os dados em treino e teste
        train_loja_item, test_loja_item = split_data_prophet(df_loja_item_grouped)
        
        # Calcular o tempo de treino
        start_train_time = time.time()
        model_loja_item = train_model_prophet(train_loja_item)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time
        tempos_treinamento[(loja, item)] = train_time
        
        # Calcular o tempo de previsão
        start_forecast_time = time.time()
        forecast = make_forecast_prophet(model_loja_item, periods=len(test_loja_item))
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time
        tempos_previsao[(loja, item)] = forecast_time
        
        # Alinhar as previsões com os dados de teste
        forecast_test_loja_item = forecast.iloc[-len(test_loja_item):].copy()
        forecast_test_loja_item['ds'] = test_loja_item['ds'].values
        
        # Armazenar o modelo treinado no dicionário
        modelos_por_loja_item[(loja, item)] = model_loja_item
        
        # Adicionar os resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_loja_item['ds'],
            'Loja': loja,
            'Item': item,
            'Real': test_loja_item['y'],
            'Previsto': forecast_test_loja_item['yhat']
        })])
        
        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)
        
        # Adicionar traces para treino, teste e previsão no gráfico da loja
        fig_loja.add_trace(go.Scatter(
            x=train_loja_item['ds'], y=train_loja_item['y'],
            mode='lines', name=f'Train (Item {item})', line=dict(color='blue')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=test_loja_item['ds'], y=test_loja_item['y'],
            mode='lines', name=f'Test (Item {item})', line=dict(color='orange')
        ), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(
            x=forecast_test_loja_item['ds'], y=forecast_test_loja_item['yhat'],
            mode='lines', name=f'Forecast (Item {item})', line=dict(color='green')
        ), row=idx+1, col=1)
        
        # Atualizar layout para cada subplot
        fig_loja.update_xaxes(title_text='Date', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Sales', row=idx+1, col=1)
    
    # Atualizar layout para a figura completa da loja
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),  # Ajusta o tamanho de acordo com a quantidade de itens
        showlegend=False,
        template='plotly_white'
    )
    
    # Salvar e exibir o gráfico para a loja
    fig_loja.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Olist/Item/prophet.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Olist/Item/prophet.csv', index=False)

# Imprimir tempos de treinamento e previsão
print("Tempos de treinamento por loja e item:", tempos_treinamento)
print("Tempos de previsão por loja e item:", tempos_previsao)

15:39:56 - cmdstanpy - INFO - Chain [1] start processing
15:39:57 - cmdstanpy - INFO - Chain [1] done processing
15:39:57 - cmdstanpy - INFO - Chain [1] start processing
15:39:57 - cmdstanpy - INFO - Chain [1] done processing


15:39:57 - cmdstanpy - INFO - Chain [1] start processing
15:39:58 - cmdstanpy - INFO - Chain [1] done processing
15:39:58 - cmdstanpy - INFO - Chain [1] start processing
15:39:58 - cmdstanpy - INFO - Chain [1] done processing


15:39:58 - cmdstanpy - INFO - Chain [1] start processing
15:39:59 - cmdstanpy - INFO - Chain [1] done processing
15:39:59 - cmdstanpy - INFO - Chain [1] start processing
15:39:59 - cmdstanpy - INFO - Chain [1] done processing


15:39:59 - cmdstanpy - INFO - Chain [1] start processing
15:40:00 - cmdstanpy - INFO - Chain [1] done processing
15:40:00 - cmdstanpy - INFO - Chain [1] start processing
15:40:00 - cmdstanpy - INFO - Chain [1] done processing


15:40:00 - cmdstanpy - INFO - Chain [1] start processing
15:40:01 - cmdstanpy - INFO - Chain [1] done processing
15:40:01 - cmdstanpy - INFO - Chain [1] start processing
15:40:01 - cmdstanpy - INFO - Chain [1] done processing


...

15:40:02 - cmdstanpy - INFO - Chain [1] start processing
15:40:03 - cmdstanpy - INFO - Chain [1] done processing
15:40:03 - cmdstanpy - INFO - Chain [1] start processing
15:40:03 - cmdstanpy - INFO - Chain [1] done processing


15:40:04 - cmdstanpy - INFO - Chain [1] start processing
15:40:04 - cmdstanpy - INFO - Chain [1] done processing
15:40:04 - cmdstanpy - INFO - Chain [1] start processing
15:40:05 - cmdstanpy - INFO - Chain [1] done processing


15:40:05 - cmdstanpy - INFO - Chain [1] start processing
15:40:05 - cmdstanpy - INFO - Chain [1] done processing
15:40:05 - cmdstanpy - INFO - Chain [1] start processing
15:40:06 - cmdstanpy - INFO - Chain [1] done processing


15:40:06 - cmdstanpy - INFO - Chain [1] start processing
15:40:07 - cmdstanpy - INFO - Chain [1] done processing
15:40:07 - cmdstanpy - INFO - Chain [1] start processing
15:40:07 - cmdstanpy - INFO - Chain [1] done processing


Tempos de treinamento por loja e item: {('6560211a19b47992c3666cc44a7e94c0', 'a92930c327948861c015c919a0bcb4a8'): 0.5665926933288574, ('6560211a19b47992c3666cc44a7e94c0', 'd017a2151d543a9885604dc62a3d9dcc'): 0.34583616256713867, ('4a3ca9315b744ce9f8e9374361493884', '99a4788cb24856965c36a24e339b6058'): 0.35188817977905273, ('4a3ca9315b744ce9f8e9374361493884', '35afc973633aaeb6b877ff57b2793310'): 0.3403918743133545, ('1f50f920176fa81dab994f9023523100', '422879e10f46682990de24d770e7f83d'): 0.5899338722229004, ('1f50f920176fa81dab994f9023523100', '389d119b48cf3043d311335e499d9c6b'): 0.41717004776000977, ('cc419e0650a3c5ba77189a1882b7556a', '154e7e31ebfa092203795c972e5804a6'): 0.29970431327819824, ('cc419e0650a3c5ba77189a1882b7556a', '2b4609f8948be18874494203496bc318'): 0.33430981636047363, ('da8622b14eb17ae2831f4ac5b9dab84a', 'e0cf79767c5b016251fe139915c59a26'): 0.5517821311950684, ('da8622b14eb17ae2831f4ac5b9dab84a', 'fc1d8637c0268af3db482c14b7ef8e75'): 0.45090770721435547, ('955fee9216a6